In [1]:
from jenga.tasks.income import IncomeEstimationTask
from jenga.corruptions.generic import MissingValues
from jenga.evaluation.corruption_impact import CorruptionImpactEvaluator
import matplotlib.pyplot as plt

import numpy as np
import datawig 

### Instantiate the income estimation task, make it ignore incomplete records for model training

In [2]:
task = IncomeEstimationTask(seed=42, ignore_incomplete_records_for_training=True)

### A missing value imputer which imputes the most frequent value per column

In [3]:
class ModeImputer:
    def __init__(self, columns):
        self.columns = columns
        self.modes = {}
        
    def fit(self, data):
        for column in self.columns:
            mode = data[column].value_counts().index[0]
            self.modes[column] = mode
            
    def transform(self, data):
        imputed = data.copy(deep=True)
        for column in self.columns:
            imputed[column].fillna(self.modes[column], inplace=True) 
        return imputed
            
imputer = ModeImputer(['education', 'workclass', 'marital_status', 'occupation'])
imputer.fit(task.train_data)

### A missing value which learns an imputation model via the datawig library

In [5]:
class DatawigImputer:
    def __init__(self, input_columns, target_column):
        self.input_columns = input_columns
        self.target_column = target_column
        self.model = None
        
    def fit(self, data):
        self.model = datawig.SimpleImputer(
            input_columns=self.input_columns,
            output_column=self.target_column,
            output_path = f'imputer_model_{self.target_column}',
            is_explainable=False).fit(data)
        self.model = self.model.load(f'imputer_model_{self.target_column}')
            
    def transform(self, data):
        imputed = data
        imputed = self.model.predict(imputed, inplace=True)
        imputed.loc[imputed[self.target_column].isnull(), self.target_column] = \
            imputed[self.target_column + '_imputed']
        
        return imputed

### We train imputation models for each column of interest

In [6]:
education_imputer = DatawigImputer(['occupation', 'marital_status', 'workclass'], 'education')
education_imputer.fit(task.train_data)

2020-10-19 19:39:05,883 [INFO]  CategoricalEncoder for column education                                found only 34 occurrences of value Preschool
2020-10-19 19:39:09,922 [INFO]  
========== start: fit model
2020-10-19 19:39:09,923 [WARNING]  Already bound, ignoring bind()
2020-10-19 19:39:13,999 [INFO]  Epoch[0] Batch [0-680]	Speed: 2679.75 samples/sec	cross-entropy=1.743749	education-accuracy=0.389776
2020-10-19 19:39:18,054 [INFO]  Epoch[0] Train-cross-entropy=1.680336
2020-10-19 19:39:18,055 [INFO]  Epoch[0] Train-education-accuracy=0.394543
2020-10-19 19:39:18,056 [INFO]  Epoch[0] Time cost=8.131
2020-10-19 19:39:18,072 [INFO]  Saved checkpoint to "imputer_model_education/model-0000.params"
2020-10-19 19:39:18,711 [INFO]  Epoch[0] Validation-cross-entropy=1.691406
2020-10-19 19:39:18,712 [INFO]  Epoch[0] Validation-education-accuracy=0.393626
2020-10-19 19:39:22,807 [INFO]  Epoch[1] Batch [0-680]	Speed: 2660.46 samples/sec	cross-entropy=1.664310	education-accuracy=0.396659
2020-1

In [ ]:
occupation_imputer = DatawigImputer(['education', 'marital_status', 'workclass'], 'occupation')
occupation_imputer.fit(task.train_data)

In [ ]:
marital_status_imputer = DatawigImputer(['education', 'occupation', 'workclass'], 'marital_status')
marital_status_imputer.fit(task.train_data)

In [ ]:
workclass_imputer = DatawigImputer(['education', 'occupation', 'marital_status'], 'workclass')
workclass_imputer.fit(task.train_data)

### Some glue code (decorators) to be able to apply the imputers in our task

In [ ]:
class ChainedModelDecorator:
    def __init__(self, model, imputers):
        self.model = model
        self.imputers = imputers
        
    def predict_proba(self, data):
        imputed = data
        for imputer in self.imputers:
            imputed = imputer.transform(imputed)
        
        return self.model.predict_proba(imputed)

In [ ]:
class ModelDecorator:
    def __init__(self, model, imputer):
        self.model = model
        self.imputer = imputer
        
    def predict_proba(self, data):
        return self.model.predict_proba(self.imputer.transform(data))

### We generate the data corruptions to evaluate: missing values of different kinds and strengths for the columns of interest

In [11]:
evaluator = CorruptionImpactEvaluator(task)

corruptions = []
for impacted_column in ['education', 'workclass', 'marital_status', 'occupation']:
    for fraction in [0.99, 0.5, 0.25, 0.1, 0.01]:
        for missingness in ['MCAR', 'MAR', 'MNAR']:
            corruption = MissingValues(impacted_column, fraction, missingness=missingness, na_value=np.nan)
            corruptions.append(corruption)

### Train the baseline model

In [12]:
model = task.fit_baseline_model(task.train_data, task.train_labels)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    2.8s finished


### We create two additional models, one that uses the datawig imputers and another one that uses the  mode imputer

In [13]:
datawig_model = ChainedModelDecorator(model, [education_imputer, workclass_imputer, 
                                                        marital_status_imputer, occupation_imputer])
mode_model = ModelDecorator(model, imputer)

### Evaluate the impact of the corruptions on the baseline model and the model with mode imputation

In [14]:
results = evaluator.evaluate(model, 10, *corruptions)
decorated_results = evaluator.evaluate(mode_model, 10, *corruptions)

0/600 (0.018067999999971107)
10/600 (0.18157499999995252)
20/600 (0.37378000000001066)
30/600 (0.571939000000043)
40/600 (0.7400380000000268)
50/600 (0.9264269999999897)
60/600 (1.1226120000000037)
70/600 (1.2921609999999646)
80/600 (1.4886249999999563)
90/600 (1.6762169999999514)
100/600 (1.8550400000000309)
110/600 (2.0449240000000373)
120/600 (2.225684000000001)
130/600 (2.391795000000002)
140/600 (2.574662999999987)
150/600 (2.7638130000000274)
160/600 (2.92689699999994)
170/600 (3.121667000000002)
180/600 (3.308360999999991)
190/600 (3.4760880000000043)
200/600 (3.659656000000041)
210/600 (3.8413570000000163)
220/600 (4.005374999999958)
230/600 (4.188624000000004)
240/600 (4.371089999999981)
250/600 (4.543155999999954)
260/600 (4.731244999999944)
270/600 (4.907306999999946)
280/600 (5.071069999999963)
290/600 (5.256968000000029)
300/600 (5.436839999999961)
310/600 (5.6028219999999465)
320/600 (5.793888000000038)
330/600 (5.985816999999997)
340/600 (6.155250000000024)
350/600 (6.34

### Evaluate the impact of the corruptions on the model with datawig imputation

In [15]:
datawig_results = evaluator.evaluate(datawig_model, 10, *corruptions)

0/600 (11.86254299999996)
10/600 (128.9645129999999)
20/600 (246.4008859999999)
30/600 (366.53010300000005)
40/600 (482.4960040000001)
50/600 (600.3487640000001)
60/600 (716.725077)
70/600 (832.312702)
80/600 (947.9203749999999)
90/600 (1065.9548650000002)
100/600 (1181.7533119999998)
110/600 (1297.347561)
120/600 (1413.7665379999999)
130/600 (1529.893301)
140/600 (1646.086065)
150/600 (1762.0853940000002)
160/600 (1875.9627460000002)
170/600 (1989.705394)
180/600 (2103.590079)
190/600 (2217.573888)
200/600 (2330.545918)
210/600 (2444.363561)
220/600 (2557.689918)
230/600 (2670.972827)
240/600 (2784.29665)
250/600 (2896.343718)
260/600 (3008.512935)
270/600 (3120.6813859999997)
280/600 (3232.7115980000003)
290/600 (3344.94825)
300/600 (3456.9909599999996)
310/600 (3565.049216)
320/600 (3673.431373)
330/600 (3781.2083070000003)
340/600 (3890.7381840000003)
350/600 (4000.4320970000003)
360/600 (4111.335174)
370/600 (4228.531938)
380/600 (4348.022967000001)
390/600 (4467.6074100000005)
40

KeyboardInterrupt: 

### Code to plot the results

In [ ]:
def find_result(column, fraction, missingness, results):
    for result in results:
        corr = result.corruption
        if corr.column == column and corr.fraction == fraction and corr.missingness == missingness:
            return result

In [ ]:
def plot_impact(column, plt, results, suffix=''):
    ax = plt.gca()
    
    scores = []
    labels = []

    for impacted_column in [column]:
        for fraction in [0.01, 0.1, 0.5, 0.99]:  
            for missingness in ['MNAR', 'MAR', 'MCAR']:                    
                result = find_result(impacted_column, fraction, missingness, results)
                scores.append(result.corrupted_scores)
                labels.append(f"{missingness} {int(fraction*100)}%")

    baseline_score = result.baseline_score            

    ax.axhline(baseline_score, linestyle='--', color='red')
    bplot = ax.boxplot(scores, showfliers=False, patch_artist=True, medianprops={'color':'black'})

    colors = ['#1e4052', '#dc6082', '#e1a677',
              '#1e4052', '#dc6082', '#e1a677', 
              '#1e4052', '#dc6082', '#e1a677', 
              '#1e4052', '#dc6082', '#e1a677']
    
    for patch, color in zip(bplot['boxes'], colors):
        patch.set_facecolor(color)
        
    ax.yaxis.grid(True)

    ax.set_xticklabels(labels)
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)
    
    ax.set_ylim((0.79, 0.895))
    ax.set_title(f"Missing values in '{column}'", fontsize=24)
    ax.tick_params(axis='both', which='major', labelsize=22)
    ax.tick_params(axis='both', which='minor', labelsize=22)    
    ax.set_ylabel('AUC', fontsize=24)
    
    plt.gcf().set_size_inches(8, 6)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_impact('education', plt, results)

In [ ]:
plot_impact('education', plt, decorated_results, '-mode')

In [ ]:
plot_impact('education', plt, datawig_results, '-datawig')

In [ ]:
plot_impact('workclass', plt, results)

In [ ]:
plot_impact('workclass', plt, decorated_results, '-mode')

In [ ]:
plot_impact('workclass', plt, datawig_results, '-datawig')

In [ ]:
plot_impact('marital_status', plt, results)

In [ ]:
plot_impact('marital_status', plt, decorated_results, '-mode')

In [ ]:
plot_impact('marital_status', plt, datawig_results, '-datawig')

In [ ]:
plot_impact('occupation', plt, results)

In [ ]:
plot_impact('occupation', plt, decorated_results, '-mode')

In [ ]:
plot_impact('occupation', plt, datawig_results, '-datawig')

### Save the results for later analysis

In [ ]:
import jsonpickle

with open("datawig-results.jsonpickle", "w") as text_file:
    text_file.write(jsonpickle.encode(datawig_results))  
    
with open("mode-results.jsonpickle", "w") as text_file:
    text_file.write(jsonpickle.encode(decorated_results))    
    
with open("no-results.jsonpickle", "w") as text_file:
    text_file.write(jsonpickle.encode(results))        